In [1]:


#First of all we need to load in the pandas library...
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt



In [30]:
from pandas_datareader import wb
matches = wb.search('SP.POP')
matches

,id,name,unit,source,sourceNote,sourceOrganization,topics


In [31]:
dat = wb.download(
    #indicator= "SP.DYN.CBRT.IN", # Birth rate, crude (per 1,000 people)","SP.DYN.CBRT.IN",
    # indicator= "SP.DYN.CDRT.IN". # Death rate, crude (per 1,000 people)","SP.DYN.CDRT.IN"
    #indicator="SP.POP.AG00.FE.IN",
    #indicator="SP.POP.TOTL",
    indicator=["SP.POP.TOTL", "SP.DYN.CBRT.IN", "SP.POP.4549.MA"],
    country="ALL",
    start=1800,
    end=2022,
)
dat


SP.POP.TOTL  SP.DYN.CBRT.IN  SP.POP.4549.MA
country                     year                                             
Africa Eastern and Southern 2021  694665117.0             NaN      12592468.0
                            2020  677243299.0       33.092414      12163439.0
                            2019  660046272.0       33.491269      11720436.0
                            2018  643090131.0       33.913034      11277635.0
                            2017  626392880.0       34.357747      10844283.0
...                                       ...             ...             ...
Czech Republic              1964          NaN             NaN        193468.0
                            1963          NaN             NaN        223631.0
                            1962          NaN             NaN        267974.0
                            1961          NaN             NaN        309977.0
                            1960          NaN             NaN        340608.0

[16554 rows x 3 columns]

In [28]:
dat['SP.DYN.CBRT.IN']

MultiIndex([('Africa Eastern and Southern', '2021'),
            ('Africa Eastern and Southern', '2020'),
            ('Africa Eastern and Southern', '2019'),
            ('Africa Eastern and Southern', '2018'),
            ('Africa Eastern and Southern', '2017'),
            ('Africa Eastern and Southern', '2016'),
            ('Africa Eastern and Southern', '2015'),
            ('Africa Eastern and Southern', '2014'),
            ('Africa Eastern and Southern', '2013'),
            ('Africa Eastern and Southern', '2012'),
            ...
            (                   'Zimbabwe', '1969'),
            (                   'Zimbabwe', '1968'),
            (                   'Zimbabwe', '1967'),
            (                   'Zimbabwe', '1966'),
            (                   'Zimbabwe', '1965'),
            (                   'Zimbabwe', '1964'),
            (                   'Zimbabwe', '1963'),
            (                   'Zimbabwe', '1962'),
            (                 

In [21]:
countries_df = wb.get_countries()
countries_df['country'] = countries_df['name']
countries_df.set_index('country', inplace=True)
countries_ds = xr.Dataset(coords=countries_df)
countries_ds.coords.update({'year': np.arange(1800, 2200), 'country': countries_ds.coords['name'].astype(str)})
countries_ds

<xarray.Dataset>
Dimensions:      (country: 299, year: 400)
Coordinates:
  * country      (country) <U80 'Aruba' ... 'Zimbabwe'
    iso3c        (country) object 'ABW' 'AFE' 'AFG' 'AFR' ... 'ZAF' 'ZMB' 'ZWE'
    iso2c        (country) object 'AW' 'ZH' 'AF' 'A9' ... 'YE' 'ZA' 'ZM' 'ZW'
    name         (country) object 'Aruba' ... 'Zimbabwe'
    region       (country) object 'Latin America & Caribbean ' ... 'Sub-Sahar...
    adminregion  (country) object '' ... 'Sub-Saharan Africa (excluding high ...
    incomeLevel  (country) object 'High income' ... 'Lower middle income'
    lendingType  (country) object 'Not classified' 'Aggregates' ... 'Blend'
    capitalCity  (country) object 'Oranjestad' '' 'Kabul' ... 'Lusaka' 'Harare'
    longitude    (country) float64 -70.02 nan 69.18 nan ... 28.19 28.29 31.07
    latitude     (country) float64 12.52 nan 34.52 nan ... -25.75 -15.4 -17.83
  * year         (year) int64 1800 1801 1802 1803 1804 ... 2196 2197 2198 2199
Data variables:
    *empty*

In [5]:
pop_ds = xr.Dataset.from_dataframe(dat)
pop_ds.coords['year'] = pop_ds.coords['year'].astype(int)
pop_ds.coords['country'] = pop_ds.coords['country'].astype(str)
pop_ds

<xarray.Dataset>
Dimensions:      (country: 266, year: 62)
Coordinates:
  * country      (country) <U52 'Afghanistan' ... 'Zimbabwe'
  * year         (year) int64 1960 1961 1962 1963 1964 ... 2018 2019 2020 2021
Data variables:
    SP.POP.TOTL  (country, year) float64 8.997e+06 9.169e+06 ... 1.509e+07

In [6]:
all_ds = xr.merge([countries_ds, pop_ds])
all_ds 

<xarray.Dataset>
Dimensions:      (country: 301, year: 400)
Coordinates:
  * country      (country) <U80 'Afghanistan' 'Africa' ... 'Zambia' 'Zimbabwe'
  * year         (year) int64 1800 1801 1802 1803 1804 ... 2196 2197 2198 2199
    iso3c        (country) object 'AFG' 'AFR' 'AFE' 'AFW' ... 'YEM' 'ZMB' 'ZWE'
    iso2c        (country) object 'AF' 'A9' 'ZH' 'ZI' ... '1W' 'YE' 'ZM' 'ZW'
    name         (country) object 'Afghanistan' 'Africa' ... 'Zambia' 'Zimbabwe'
    region       (country) object 'South Asia' ... 'Sub-Saharan Africa '
    adminregion  (country) object 'South Asia' ... 'Sub-Saharan Africa (exclu...
    incomeLevel  (country) object 'Low income' ... 'Lower middle income'
    lendingType  (country) object 'IDA' 'Aggregates' ... 'IDA' 'Blend'
    capitalCity  (country) object 'Kabul' '' '' ... "Sana'a" 'Lusaka' 'Harare'
    longitude    (country) float64 69.18 nan nan nan ... nan 44.21 28.29 31.07
    latitude     (country) float64 34.52 nan nan nan ... nan 15.35 -15.4 -17.83
Data variables:
    SP.POP.TOTL  (country, year) float64 nan nan nan nan nan ... nan nan nan nan

In [7]:
all_ds['SP.POP.TOTL'].sel(country='France', year=2015)

<xarray.DataArray 'SP.POP.TOTL' ()>
array(66548272.)
Coordinates:
    country      <U80 'France'
    year         int64 2015
    iso3c        object 'FRA'
    iso2c        object 'FR'
    name         object 'France'
    region       object 'Europe & Central Asia'
    adminregion  object ''
    incomeLevel  object 'High income'
    lendingType  object 'Not classified'
    capitalCity  object 'Paris'
    longitude    float64 2.351
    latitude     float64 48.86

In [10]:
%matplotlib qt
fig, ax = plt.subplots()
for c in countries_ds.coords['country'].to_numpy()[::30]:
    ax.plot(all_ds['year'], all_ds['SP.POP.TOTL'].sel(country=c), label=c)
ax.legend()
plt.show()

In [9]:
countries_ds['swag'].sel(country=c)

KeyError: 'swag'

In [33]:
df = pd.read_csv("/home/lionel/pypulations/data/Population-EstimatesData.csv")
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2042,2043,2044,2045,2046,2047,2048,2049,2050,Unnamed: 95
0,Africa Eastern and Southern,AFE,Age dependency ratio (% of working-age populat...,SP.POP.DPND,88.055694,89.005350,89.751390,90.300930,90.638094,90.769594,...,6.381181e+01,6.322999e+01,6.264905e+01,6.207545e+01,6.159779e+01,6.111044e+01,6.061999e+01,6.013323e+01,5.965552e+01,NaN
1,Africa Eastern and Southern,AFE,"Age dependency ratio, old",SP.POP.DPND.OL,5.805951,5.841215,5.854827,5.850969,5.833747,5.805963,...,7.157719e+00,7.277111e+00,7.404261e+00,7.542365e+00,7.697824e+00,7.857024e+00,8.020667e+00,8.188622e+00,8.361687e+00,NaN
2,Africa Eastern and Southern,AFE,"Age dependency ratio, young",SP.POP.DPND.YG,82.046443,82.943915,83.658328,84.190146,84.522152,84.663386,...,5.618336e+01,5.550079e+01,5.481238e+01,5.412144e+01,5.350793e+01,5.288174e+01,5.224811e+01,5.161340e+01,5.098178e+01,NaN
3,Africa Eastern and Southern,AFE,"Age population, age 00, female, interpolated",SP.POP.AG00.FE.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Eastern and Southern,AFE,"Age population, age 00, male, interpolated",SP.POP.AG00.MA.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49737,Zimbabwe,ZWE,Rural population growth (annual %),SP.RUR.TOTL.ZG,NaN,3.098226,3.078315,2.823469,2.794387,2.757117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49738,Zimbabwe,ZWE,Sex ratio at birth (male births per female bir...,SP.POP.BRTH.MF,NaN,NaN,1.030000,NaN,NaN,NaN,...,1.021000e+00,NaN,NaN,NaN,NaN,1.021000e+00,NaN,NaN,NaN,NaN
49739,Zimbabwe,ZWE,Urban population,SP.URB.TOTL,476164.000000,500665.000000,528409.000000,567387.000000,609177.000000,653686.000000,...,8.754000e+06,9.026000e+06,9.301000e+06,9.580000e+06,9.860000e+06,1.014300e+07,1.042800e+07,1.071500e+07,1.100300e+07,NaN
49740,Zimbabwe,ZWE,Urban population (% of total population),SP.URB.TOTL.IN.ZS,12.608000,12.821000,13.082000,13.578000,14.092000,14.620000,...,4.068900e+01,4.133700e+01,4.198900e+01,4.264300e+01,4.329900e+01,4.395800e+01,4.461900e+01,4.528200e+01,4.594700e+01,NaN


In [37]:
#df["Indicator Code"].unique()
df["Country Name"].unique()

array(['SP.POP.DPND', 'SP.POP.DPND.OL', 'SP.POP.DPND.YG',
       'SP.POP.AG00.FE.IN', 'SP.POP.AG00.MA.IN', 'SP.POP.AG01.FE.IN',
       'SP.POP.AG01.MA.IN', 'SP.POP.AG02.FE.IN', 'SP.POP.AG02.MA.IN',
       'SP.POP.AG03.FE.IN', 'SP.POP.AG03.MA.IN', 'SP.POP.AG04.FE.IN',
       'SP.POP.AG04.MA.IN', 'SP.POP.AG05.FE.IN', 'SP.POP.AG05.MA.IN',
       'SP.POP.AG06.FE.IN', 'SP.POP.AG06.MA.IN', 'SP.POP.AG07.FE.IN',
       'SP.POP.AG07.MA.IN', 'SP.POP.AG08.FE.IN', 'SP.POP.AG08.MA.IN',
       'SP.POP.AG09.FE.IN', 'SP.POP.AG09.MA.IN', 'SP.POP.AG10.FE.IN',
       'SP.POP.AG10.MA.IN', 'SP.POP.AG11.FE.IN', 'SP.POP.AG11.MA.IN',
       'SP.POP.AG12.FE.IN', 'SP.POP.AG12.MA.IN', 'SP.POP.AG13.FE.IN',
       'SP.POP.AG13.MA.IN', 'SP.POP.AG14.FE.IN', 'SP.POP.AG14.MA.IN',
       'SP.POP.AG15.FE.IN', 'SP.POP.AG15.MA.IN', 'SP.POP.AG16.FE.IN',
       'SP.POP.AG16.MA.IN', 'SP.POP.AG17.FE.IN', 'SP.POP.AG17.MA.IN',
       'SP.POP.AG18.FE.IN', 'SP.POP.AG18.MA.IN', 'SP.POP.AG19.FE.IN',
       'SP.POP.AG19.MA.IN', 'SP.